In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CS406

/content/drive/MyDrive/CS406


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import cv2
import random
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import keras
from keras import layers

import tensorflow as tf

In [ ]:
"""
# Hàm để áp dụng histogram equalization lên ảnh
def apply_histogram_equalization(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    img_equalized = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    return img_equalized
"""
def apply_histogram_equalization(image_path):
    # Đọc ảnh
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Chuyển ảnh từ BGR sang YUV
    #img_yuv = cv2.cvtColor(image_path, cv2.COLOR_BGR2YUV)
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    # Tạo đối tượng CLAHE
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))

    # Áp dụng CLAHE trên kênh Y (độ sáng)
    img_yuv[:, :, 0] = clahe.apply(img_yuv[:, :, 0])

    # Chuyển ngược về BGR
    img_clahe = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    return img_clahe


In [ ]:
import cv2
import numpy as np

def reduce_brightness(image_path):
    """
    Giảm độ sáng của ảnh bằng cách giảm giá trị pixel đồng đều.

    Args:
        image_path (str): Đường dẫn đến ảnh.
        brightness_factor (float): Hệ số giảm sáng (0.0 - 1.0). Giá trị nhỏ hơn tạo ảnh tối hơn.

    Returns:
        numpy.ndarray: Ảnh đã được giảm sáng.
    """
    brightness_factor=0.2
    # Đọc ảnh
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Chuyển ảnh sang kiểu float để thực hiện giảm sáng chính xác
    img_float = img.astype(np.float32)

    # Giảm giá trị pixel bằng hệ số
    img_darker = img_float * brightness_factor

    # Clip giá trị để nằm trong khoảng hợp lệ (0-255)
    img_darker = np.clip(img_darker, 0, 255)

    # Chuyển ảnh về kiểu uint8
    img_darker = img_darker.astype(np.uint8)

    return img_darker


In [ ]:
# Hàm lưu ảnh và nhãn đã xử lý sang thư mục mới
def process_and_save_images(image_folder, label_folder, set_type):
    images = [img for img in os.listdir(image_folder)]
    labels = [label for label in os.listdir(label_folder)]
    print(len(labels))
    for img_name in images:
        img_path = os.path.join(image_folder, img_name)
        label_name = img_name.rsplit('.', 1)[0] + ".txt"  # Tên nhãn tương ứng với ảnh

        # Kiểm tra xem nhãn có tồn tại không
        if os.path.exists(os.path.join(label_folder, label_name)):
            # Tăng cường ảnh bằng hàm apply_histogram_equalization, khi thực hiện giảm sáng để có dataset ảnh tối thì xài reduce_brightness
            enhanced_image = apply_histogram_equalization(img_path)

            # Chuyển đổi sang định dạng RGB và thay đổi kích thước
            enhanced_image = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB)
            enhanced_image = Image.fromarray(enhanced_image).resize((new_size, new_size))

            # Lưu ảnh đã tăng cường
            enhanced_image.save(os.path.join(Enhanced_Dataset_path, set_type, "images", img_name))
            """
            # Sao chép nhãn mà không chỉnh sửa bounding box
            label_path = os.path.join(label_folder, label_name)
            with open(label_path, 'r') as label_file:
                lines = label_file.readlines()

            # Lưu nhãn vào thư mục tương ứng
            with open(os.path.join(Enhanced_Dataset_path, set_type, "labels", label_name), 'w') as new_label_file:
                new_label_file.writelines(lines)
            """

In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt

def display_random_images(image_folder, num_images=10):
    """
    Hiển thị ảnh gốc, ảnh giảm sáng, và ảnh enhanced từ thư mục cho trước.

    Args:
        image_folder (str): Đường dẫn thư mục chứa ảnh.
        num_images (int): Số lượng ảnh muốn hiển thị.
    """
    # Lấy danh sách các ảnh trong thư mục
    images = [img for img in os.listdir(image_folder) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    random_images = random.sample(images, min(num_images, len(images)))

    # Tạo figure để hiển thị ảnh
    plt.figure(figsize=(15, num_images * 5))

    for i, img_name in enumerate(random_images):
        img_path = os.path.join(image_folder, img_name)

        # Ảnh gốc
        original_image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

        # Ảnh giảm sáng
        reduce_brightness_image = reduce_brightness(img_path)
        reduce_brightness_image_rgb = cv2.cvtColor(reduce_brightness_image, cv2.COLOR_BGR2RGB)

        # Ảnh enhanced
        enhanced_image = apply_histogram_equalization(reduce_brightness_image_rgb)
        enhanced_image_rgb = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2RGB)

        # Hiển thị ảnh gốc
        plt.subplot(num_images, 3, 3 * i + 1)
        plt.imshow(original_image_rgb)
        plt.title(f"Original Image {i+1}")
        plt.axis('off')

        # Hiển thị ảnh giảm sáng
        plt.subplot(num_images, 3, 3 * i + 2)
        plt.imshow(reduce_brightness_image_rgb)
        plt.title(f"Reduce Brightness {i+1}")
        plt.axis('off')

        # Hiển thị ảnh enhanced
        plt.subplot(num_images, 3, 3 * i + 3)
        plt.imshow(enhanced_image_rgb)
        plt.title(f"Enhanced Image {i+1}")
        plt.axis('off')

    # Tối ưu hóa layout
    plt.tight_layout()
    plt.show()


In [ ]:
# Định nghĩa đường dẫn và các thông số
Dataset_path = "/content/drive/MyDrive/CS406/Dark_Dataset"
Enhanced_Dataset_path = "/content/drive/MyDrive/CS406/Enhanced_Dataset"
new_size = 640  # Giữ kích thước ảnh là 640x640
# Danh sách các thư mục ảnh để xử lý
image_folder_train = os.path.join(Dataset_path, "train", "images")
label_folder_train = os.path.join(Dataset_path, "train", "labels")
image_folder_val = os.path.join(Dataset_path, "val", "images")
label_folder_val = os.path.join(Dataset_path, "val", "labels")
image_folder_test = os.path.join(Dataset_path, "test", "images")
label_folder_test = os.path.join(Dataset_path, "test", "labels")



In [ ]:

# Gọi hàm để in ra 10 ảnh ngẫu nhiên từ tập train
display_random_images(image_folder_train, num_images=20)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Xử lý và lưu ảnh và nhãn cho các bộ train, val, test
process_and_save_images(image_folder_train, label_folder_train, "train")
process_and_save_images(image_folder_val, label_folder_val, "val")
process_and_save_images(image_folder_test, label_folder_test, "test")

print("Hoàn thành lưu ảnh và nhãn tăng cường vào các thư mục Train, Val, Test.")

2099
600
300
Hoàn thành lưu ảnh và nhãn tăng cường vào các thư mục Train, Val, Test.


In [ ]:

import os
import xml.etree.ElementTree as ET

# Thông tin kích thước ảnh sau resize
image_width = 256
image_height = 256

# Hàm chuyển đổi YOLO sang VOC XML
def yolo_to_voc(yolo_label_path, image_folder, output_voc_folder):
    for label_file in os.listdir(yolo_label_path):
        if label_file.endswith(".txt"):
            label_path = os.path.join(yolo_label_path, label_file)
            # Lấy tên file mà không có đuôi .txt
            image_name_without_ext = label_file.replace(".txt", "")
            # Tìm ảnh có tên tương ứng (có thể có đuôi khác nhau)
            image_path = None
            for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.PNG','.JPEG']:
                potential_image_path = os.path.join(image_folder, image_name_without_ext + ext)
                if os.path.exists(potential_image_path):
                    image_path = potential_image_path
                    image_name = image_name_without_ext + ext
                    break

            # Kiểm tra nếu không tìm thấy ảnh tương ứng
            if image_path is None:
                print(f"Warning: Image for label file {label_file} not found!")
                continue

            # Tạo file XML
            annotation = ET.Element("annotation")

            # Thêm thông tin cơ bản
            ET.SubElement(annotation, "folder").text = "images"
            ET.SubElement(annotation, "filename").text = image_name
            ET.SubElement(annotation, "path").text = image_path

            size = ET.SubElement(annotation, "size")
            ET.SubElement(size, "width").text = str(image_width)
            ET.SubElement(size, "height").text = str(image_height)
            ET.SubElement(size, "depth").text = "3"

            ET.SubElement(annotation, "segmented").text = "0"

            # Đọc file YOLO label
            with open(label_path, "r") as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                class_id = parts[0]
                x_center, y_center, width, height = map(float, parts[1:])

                # Chuyển đổi sang tọa độ VOC
                xmin = int((x_center - width / 2) * image_width)
                ymin = int((y_center - height / 2) * image_height)
                xmax = int((x_center + width / 2) * image_width)
                ymax = int((y_center + height / 2) * image_height)

                # Tạo object trong XML
                obj = ET.SubElement(annotation, "object")
                ET.SubElement(obj, "name").text = f"class_{class_id}"  # Đặt tên class
                ET.SubElement(obj, "pose").text = "Unspecified"
                ET.SubElement(obj, "truncated").text = "0"
                ET.SubElement(obj, "difficult").text = "0"

                bndbox = ET.SubElement(obj, "bndbox")
                ET.SubElement(bndbox, "xmin").text = str(xmin)
                ET.SubElement(bndbox, "ymin").text = str(ymin)
                ET.SubElement(bndbox, "xmax").text = str(xmax)
                ET.SubElement(bndbox, "ymax").text = str(ymax)

            # Ghi file XML
            xml_file = os.path.join(output_voc_folder, label_file.replace(".txt", ".xml"))
            tree = ET.ElementTree(annotation)
            tree.write(xml_file)

# Cấu hình đường dẫn
base_input_path = "/content/drive/MyDrive/CS406/Dataset"
base_output_path = "/content/drive/MyDrive/CS406/Enhanced_Dataset"
output_voc_path = os.path.join(base_output_path, "VOC_Annotations")
splits = ["train", "val", "test"]
os.makedirs(output_voc_path, exist_ok=True)
for split in splits:
    input_image_path = os.path.join(base_input_path, split, "images")
    input_label_path = os.path.join(base_input_path, split, "labels")
    # Chuyển đổi YOLO sang VOC
    yolo_to_voc(input_label_path, input_image_path, output_voc_path)

print("Hoàn tất chuyển đổi tất cả YOLO -> VOC.")


Hoàn tất chuyển đổi tất cả YOLO -> VOC.
